# Predicting heart disease using machine learning
this notebook looks into using many Python-based and data science libraries in an attempt to build a machine learning model to predict heart disease. In order to run successfully the program, the following command needs to be run for having the proper env: `conda env create --prefix ./env -f environtment.yml`

In [1]:
import  numpy as np